## Task 2 
#### Track a specific vehicle in a given video

In [1]:
# Importing libraries
import cv2
import numpy as np
import os
import glob
cv2.__version__

'4.7.0'

In [2]:
# Required files paths 
Train_Mode = False

if Train_Mode:
    videos = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\train\Task2\*.mp4'
    bounding_boxes = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\train\Task2\*.txt'
else:
    videos = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\test\Task2\*.mp4'
    bounding_boxes = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\test\Task2\*.txt'
    
videos_paths = sorted(glob.glob(videos), key=lambda i: int(os.path.splitext(os.path.basename(i))[0]))
boxes_paths = sorted(glob.glob(bounding_boxes), key=lambda i: int(os.path.splitext(os.path.basename(i))[0]))

results_paths = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\Lara_Tomeh_507\Task2'
ground_truth_paths = r'D:\Uni_Buc\Uni_Buc_semester_2\Computer Vision\CV 2023\CV-2023-Project2\train\Task2\ground-truth'

In [ ]:
# # Different types of Python open cv trackers
# tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
# tracker_type = tracker_types[0]
# if tracker_type == 'BOOSTING':
#     tracker = cv2.legacy.TrackerBoosting_create()
# if tracker_type == 'MIL':
#     tracker = cv2.TrackerMIL_create() 
# if tracker_type == 'KCF':
#     tracker = cv2.TrackerKCF_create() 
# if tracker_type == 'TLD':
#     tracker = cv2.legacy.TrackerTLD_create() 
# if tracker_type == 'MEDIANFLOW':
#     tracker = cv2.legacy.TrackerMedianFlow_create() 
# # if tracker_type == 'GOTURN':
# #     tracker = cv2.TrackerGOTURN_create()
# if tracker_type == 'MOSSE':
#     tracker = cv2.legacy.TrackerMOSSE_create()
# if tracker_type == "CSRT":
#     tracker = cv2.TrackerCSRT_create()

In [3]:
# Read the actual bounding boxes
def bbox():
    all_bboxes = []
    all_frames = []
    for i, box in enumerate(boxes_paths):
        with open(box) as f:
            frames = int(f.readline().split(' ')[0])
            all_frames.append(frames)
            _, x1, y1, x2, y2 = f.readline().split()[:5]
            all_bboxes.append([int(x1), int(y1), int(x2), int(y2)])
            
    return all_frames, all_bboxes

all_frames, all_bboxes = bbox()

# Create the predicted bounding boxes dictionary
predicted_bboxes = {}
for i, video_path in enumerate(videos_paths, start=1):
    predicted_bboxes[i] = []

In [4]:
# Start tracking
for i in range(0, 15): # 15 is the total videos
    
    # Create an object of Python Tracker
    tracker = cv2.legacy.TrackerBoosting_create()
    
    # Initialize frame_number
    frame_number = 0 
    
    # Get the video file and read it   
    video = cv2.VideoCapture(videos_paths[i])
    
    # Read the first frame
    _, frame = video.read() 

    # We perform this subtraction to get the width and height from (xmin,ymin) & (xmax,ymax), top_left & bottom_right points
    [x, y, w, h] = all_bboxes[i]  
    bbox = (x, y, w-x, h-y)  

    # Append first bounding box to the list
    predicted_bboxes[i + 1].append([frame_number, int(bbox[0]), int(bbox[1]), int(bbox[0]) + int(bbox[2]), int(bbox[1]) + int(bbox[3])])

    # If we want to select the bounding box manually
    # bbox = cv2.selectROI(frame, False)
    
    # Initialize the tracker
    tracker.init(frame, bbox)  

    while True:
        ret, frame = video.read()
        if not ret:
#             print('ERROR!')
            break
            
        frame_number += 1
        
        # Update the tracker
        ret, bbox = tracker.update(frame)  
        
        if ret:
            # Draw a rectangle surrounding the object
            cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3])), (255, 255, 255), 2, 1)
        
        # Add the actual bounding box to the list
        predicted_bboxes[i + 1].append([frame_number, int(bbox[0]), int(bbox[1]), int(bbox[0]) + int(bbox[2]), int(bbox[1]) + int(bbox[3])])

        # Open the file and write the content (bounding boxes' coordinations)
        for k, v in predicted_bboxes.items():
            if not os.path.exists(results_paths):
                os.makedirs(results_paths)
            file_num = ''
            number = k
            if number < 10:
                file_num = '0' + str(number)
            else:
                file_num = str(number)
            f = open(results_paths + '/' + file_num + '_predicted.txt', 'w')
            f.write(str(all_frames[k - 1]) + ' -1 -1 -1 -1\n')
            for [a, x, y, w, h] in v:
                f.write(str(a) + ' ' + str(x) + ' ' + str(y) + ' ' + str(w) + ' ' + str(h) + '\n')
            f.close()
          
          # In case we want to show the tracking frame
#         cv2.imshow("Tracking objects", frame)
#         k = cv2.waitKey(1) & 0xff
#         if k == 27 :
#             break

video.release()       
cv2.destroyAllWindows()

In [ ]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the intersection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

def compute_percentage_tracking(gt_bboxes, predicted_bboxes, num_frames,verbose=0):
    """
    This function compute the percentage of detected bounding boxes based on the ground-truth bboxes and the predicted ones.
    :param gt_bboxes. The ground-truth bboxes with the format: frame_idx, x_min, y_min, x_max, y_max.
    :param predicted_bboxes. The predicted bboxes with the format: frame_idx, x_min, y_min, x_max, y_max
    :param num_frames. The total number of frames in the video.
    """
    
    num_frames = int(num_frames)
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    gt_dict = {}
    for gt_box in gt_bboxes:
        gt_dict[gt_box[0]] = gt_box[1:]
    
    pred_dict = {}
    for pred_bbox in predicted_bboxes:
        pred_dict[pred_bbox[0]] = pred_bbox[1:]
        
    for i in range(num_frames):
        if gt_dict.get(i, None) is None and pred_dict.get(i, None) is None: # the bowling bowl is not on the lane
            tn += 1 
        
        elif gt_dict.get(i, None) is not None and pred_dict.get(i, None) is None: # the bowling ball is not detected
            fn += 1
            
        elif gt_dict.get(i, None) is None and pred_dict.get(i, None) is not None: # the bowling ball is not on the lane, but it is 'detected'
            fp += 1
            
        elif gt_dict.get(i, None) is not None and pred_dict.get(i, None) is not None: # the bowling ball is on the lane and it is detected
            
            iou = bb_intersection_over_union(gt_dict[i], pred_dict[i])
            if iou >= 0.2:
                tp += 1
            else:
                fp += 1 
                         
    if verbose:
        print(f'tp = {tp}, tn = {tn}, fp = {fp},fn = {fn}')
    assert tn + fn + tp + fp == num_frames
    perc = (tp + tn) / (tp + fp + tn + fn)
    
    return perc

def evaluate_results_task2(predictions_path,ground_truth_path, verbose = 0):
    total_correct_tracked_videos = 0
    for i in range(1,16):

        correct_tracked_video = 0
    
        try:

            if(i<10):
                name = '0' + str(i)
            else:
                name = str(i)
    
            filename_predictions = predictions_path + "/" + name + "_predicted.txt"
            filename_ground_truth = ground_truth_path + "/" + name + "_gt.txt"

            p = np.loadtxt(filename_predictions)   
            predicted_bboxes = p[1:]
            gt = np.loadtxt(filename_ground_truth)
            gt_bboxes = gt[1:]
            num_frames = gt[0][0]
            percentage = compute_percentage_tracking(gt_bboxes, predicted_bboxes, num_frames,verbose)

            correct_tracked_video = 1
            if percentage < 0.8:
                correct_tracked_video = 0
                print(i)  
            else:
                print(percentage)
        
    
            if verbose:
                print("percentage = ", percentage)
                print("Task 2 - Tracking the vehicle: for test example number ", str(i), " the prediction is :", (1-correct_tracked_video) * "in" + "correct", "\n")
        
            total_correct_tracked_videos = total_correct_tracked_videos + correct_tracked_video
    
        except:
            print("Error")

        points = total_correct_tracked_videos * 0.1
        
    return total_correct_tracked_videos,points 

In [ ]:
print(evaluate_results_task2(results_paths, ground_truth_paths, verbose = 1))